# Teoría de la Información y la Codificación
## Tarea: Códigos de Hamming


# Autor: 

Rellene la siguiente información:

 - Estudiante (nombre y apellidos): Jorge Gangoso Klöck
 - DNI/NIE/Pasaporte: 49398653N
 - Grupo: 1
 - Curso académico: 2021-2022



__Yo, como estudiante de la asignatura, aseguro que la elaboración de estos ejercicios ha sido realizada de forma individual, sin incurrir en copias parciales o totales de código fuente o documentación, y acepto las repercusiones que conllevaría si esto no fuese así.__


# Respuestas a las cuestiones teóricas:

## Códigos de Hamming
### Definición

Los códigos de Hamming son una subcategoría dentro de los códigos lineales (Por lo que también es de longitud uniforme) y se define como un código detector y corrector de errores.
La característica que debe tener un código lineal para ser considerado código de Hamming es que éstos se construyen añadiendo bits de paridad en las posiciones $2^n$ del mensaje original, que se construyen sobre el campo de galois $GF(2)$ y que estos bits de paridad se calculan utilizando subconjuntos del mensaje original que sean linealmente independientes.


### Codificación y decodificación

Para codificar un mensaje utilizando un código de Hamming, añadimos los r bits de paridad necesarios que los calcularemos como $2^r-r-1 >= k$. Como se ha mencionado anteriormente estos bits de paridad se colocarán en posiciones que sean potencias de 2 y su valor será el necesario para mantener la paridad par dentro de un conjunto de elementos del mensaje original. Para ello tomamos los elementos del mensaje que se van a comprobar su paridad y se realiza una función "xor" o una suma módulo 2 que viene a tener el mismo resultado en $GF(2)$.
Los elementos que deben tomarse son todos los elementos de forma alterna en rachas de P, donde P es la posición del bit de paridad, es decir, el bit de paridad que se coloca en la posición 4, se calcula utilizando los elementos (a partir de él mismo) tomando 4 elementos sí, 4 elementos no, sucesivamente hasta terminar el mensaje.

Para decodificar cualquiera de estos mensajes es sencillo ver que lo único que alteramos del mensaje original es añadirle bits de paridad, por tanto basta con quitarlos del mensaje codificado para recuperar el mensaje original.


### Corrección de errores y distancia

Los códigos de Hamming aseguran detección y correción en errores de 1 bit y detección en errores de 2 bits (sin capacidad de corrección). 
El motivo de estos valores es que se parte de un código uniforme con Distancia(Código) = 1 por estar tratando con códigos uniformes y eso nos indica que el código por sí mismo no es capaz de detectar o corregir ningún error ya que como sabemos por el Teorema de Hamming necesitamos una distancia de $r + 1$ para detectar $r$ errores y de $2*r + 1$ para corregirlos.
Por ello la codificación de Hamming lo que provoca es ampliar la distancia del código en dos de forma que $D(C) = 3$.
Esto nos permite detectar $3 = p + 1$ errores, donde $p = 2$.
Y corregir $3 = 2p + 1$ errores, donde $p = 1$.

La corrección de errores, cuando se detectan correctamente, es instantánea, ya que el propio síndrome del comprobador se corresponde con la posición en binario del bit afectado. Sin embargo hemos comprobado como la corrección de errores cuando hay 2 errores no funciona correctamente, ya que el síndrome nos indica una posición y hay dos errores, con lo que en cualquier caso nunca recuperaríamos el mensaje original. Hay una forma de avisar al sistema comprobador de que hay una cantidad par de errores que es añadir un bit más de paridad que comprueba la paridad total del bloque, de esta forma si hay errores de paridad, pero el bit de paridad global es correcto, sabemos que hay una cantidad par mayor que 0 de errores, y por tanto, lo mejor es directamente descartar el mensaje y no intentar corregirlo.

In [59]:

#imports
import numpy as np
import operator as op


In [60]:

# Declaración de constantes a usar 

# p debe ser 4 para asegurar corregir errores de 1 bit en mensajes de 2^4-4-1 bits (11), con 3 sólo podríamos codificar
# mensajes de 4 bits.

r = 4;

# Longitud del código uniforme de codificación de la fuente (k)

k = 8; #Se necesitan 8 bits para codificar 256 caracteres.

# Longitud del código de bloque (n)


n = k + r; # n = 12;
#Tendremos un código de Hamming (12,8)

In [61]:


# Generación aleatoria de una palabra de k bits
# Función que tiene como entrada k, la longitud de un código, y genera
# una palabra de un código de longitud k sobre GF(2)
def GenerarPalabraAleatoria(k):
    palabra= np.random.randint(0, 2, k)
    return palabra


# Función que tiene como entrada una palabra de un código sobre GF(2) y genera un error de 1 bit en dicha palabra
def InsertarError1bit(palabra):
    pos= np.random.randint(0, len(palabra))
    palabra[pos]= 1-palabra[pos]

    
# Función que tiene como entrada una palabra de un código sobre GF(2) y genera un error de 2 bits en dicha palabra
def InsertarError2bits(palabra):
    pos1= np.random.randint(0, len(palabra))
    palabra[pos1]= 1-palabra[pos1]
    pos2= np.random.randint(0, len(palabra))
    while (pos1 == pos2):
        pos2= np.random.randint(0, len(palabra))
    palabra[pos2]= 1-palabra[pos2]



In [62]:

# Función CalcularMatrizM que tiene como entrada:
#  - k: la dimensión del código (2^k mensajes posibles a codificar)
#  - n: la longitud del código de bloque
# Da como salida una matriz de codificación de un código de Hamming (n,k)

def CalcularMatrizM (n, k):
    r = n-k;
    parity = np.ones(r).astype(int);
    parity[0] = 1;
    for x in range(1,len(parity)):
        parity[x] = 2*parity[x-1];

    #Creamos check_bit, una especie de máscara que nos dirá para cada bit de paridad, que elementos de X debemos observar
    check_bit = np.full((r,n),0, int);
    for j in range(len(parity)):
        for i in range(parity[j], parity[j]*2):
            check_bit[j][i-1::2*parity[j]] = 1;
    #Ahora quitamos del check bit las columnas que se corresponden con valores de paridad (identidad)
    indexes = [];
    for x in range(len(parity)):
        indexes = np.append(indexes, parity[x]-1);
    indexes = indexes.astype(int);

    #Eliminamos dichas columnas y volteamos la matriz para que quede con los bits menos significativos al final
    check_bit = np.delete(check_bit, indexes, axis=1);  
    check_bit = np.flip(check_bit).T;
    check_bit = np.flip(check_bit, axis = 1);

    M = np.identity(k).astype(int);
    for i in range(len(parity)):
        pos = len(M[0])-(parity[i]-1);
        M = np.insert(M, pos, check_bit[:,i], axis=1);
    return M;


In [63]:

# Función CalcularMatrizH que tiene como entrada:
#  - k: la dimensión del código (2^k mensajes posibles a codificar)
#  - n: la longitud del código de bloque
# Da como salida la matriz de comprobación de errores (H) del código de Hamming (n,k)

def CalcularMatrizH (n, k):
    r = n - k;
    n_fil = n;
    parity = np.ones(r).astype(int);
    parity[0] = 1;
    for x in range(1,len(parity)):
        parity[x] = 2*parity[x-1];

    #Creamos la matriz con las filas y columnas traspuestas para rellenarla por filas
    H = np.full((r, n_fil), 0, int);
    for i in range (r):
        for j in range (parity[i]-1,parity[i]*2-1):  
            H[i][j::parity[i]*2] = 1; 
    H = np.flip(H);
    #La trasponemos para tener la matriz H
    H = H.T;
    return H;

In [64]:

# Función Codificar que tiene como entrada:
#  - Palabra: Una palabra de longitud k sobre GF(2) a codificar
#  - M: La matriz de codificación del código de Hamming
# Devuelve como salida la palabra del código de bloque de codificar la palabra de entrada en el código de Hamming

def Codificar (palabra, M):
    word = np.matmul(palabra, M);
    word = word%2;
    return word;


In [65]:

# Función Decodificar que tiene como entrada:
#  - Palabra: Una palabra de longitud n sobre GF(2) a decodificar
#  - M: La matriz de codificación del código de Hamming
# Devuelve como salida la palabra del código uniforme original decodificada

def Decodificar (palabra_codificada, M):
    [k, n] = M.shape
    r = n-k;
    parity = np.ones(r).astype(int);
    for x in range(1,len(parity)):
        parity[x] = 2*parity[x-1];
    parity = parity - 1;
    palabra_codificada = np.flip(palabra_codificada);
    palabra_codificada = np.delete(palabra_codificada, parity);
    palabra_codificada = np.flip(palabra_codificada);
    return palabra_codificada;


In [66]:

# Función CalcularSindrome: Calcula el síndrome de una palabra del código de bloque de Hamming dada
# Se tiene como entrada:
#  - Palabra: La palabra del código de bloque de la que se desea recibir el síndrome.
#  - H: La matriz de comprobación de errores del código de hamming
# Como salida, se devuelve el síndrome asociado a la palabra de entrada

def CalcularSindrome (palabra, H):
    sindrome = np.matmul(palabra,H);
    return sindrome%2;

In [67]:

# Función CorregirError: Calcula el error asociado a una palabra dada, y lo corrige
# Se tiene como entrada:
#  - Palabra: La palabra del código de bloque que se desea corregir
#  - Sindrome: El síndrome asociado a la palabra
# Como salida, se devuelve el la palabra corregida, en caso de que hubiese error-

def CorregirError (palabra, sindrome):
    posicion = ''.join(map(str,sindrome));
    posicion = int(posicion,2)-1;
    if posicion >= 0:
        palabra = np.flip(palabra);
        palabra[posicion] = (palabra[posicion]+1)%2;
        palabra = np.flip(palabra);
    return palabra;


In [73]:
# EJEMPLO 1

# Ejemplos de prueba de envío sin errores

# Generar mensaje a enviar
palabra = GenerarPalabraAleatoria(k);
# Codificación en código de bloque
M = CalcularMatrizM(n, k);
coded_word = Codificar (palabra, M);
# Cálculo del error y corrección
H = CalcularMatrizH(n, k);
sindrome = CalcularSindrome(coded_word, H);
palabra_corregida = CorregirError(coded_word, sindrome);
# Comprobación con el mensaje del código de bloque enviado por el canal
print("## COMPARACION DEL MENSAJE CODIFICADO CON EL CORREGIDO ##");
print("palabra codificada:\t ", coded_word);
print("palabra corregida:\t ", palabra_corregida);
print("sindrome:\t\t ", sindrome);
# Decodificación.
palabra_decodificada = Decodificar (palabra_corregida, M);
# Comparación con el mensaje del código uniforme generado por la fuente
print("## COMPARACION DEL MENSAJE DECODIFICADO CON EL ORIGINAL ##");
print("palabra original:\t ", palabra);
print("palabra decodificada:\t ", palabra_decodificada);

## COMPARACION DEL MENSAJE CODIFICADO CON EL CORREGIDO ##
palabra codificada:	  [1 0 1 0 0 0 0 1 0 1 0 0]
palabra corregida:	  [1 0 1 0 0 0 0 1 0 1 0 0]
sindrome:		  [0 0 0 0]
## COMPARACION DEL MENSAJE DECODIFICADO CON EL ORIGINAL ##
palabra original:	  [1 0 1 0 0 0 1 1]
palabra decodificada:	  [1 0 1 0 0 0 1 1]


In [76]:

# EJEMPLO 2

# Ejemplos de prueba de envío con error en 1 bit

# Generar mensaje a enviar
palabra = GenerarPalabraAleatoria(k);
# Codificación en código de bloque
M = CalcularMatrizM(n, k);
coded_word = Codificar (palabra, M);
coded_word_original = coded_word.copy();
# Inserción de un error de 1 bit
InsertarError1bit(coded_word);
palabra_erronea = coded_word.copy();
# Cálculo del error y corrección
H = CalcularMatrizH(n, k);
sindrome = CalcularSindrome(coded_word, H);
palabra_corregida = CorregirError(coded_word, sindrome);
# Comprobación con el mensaje del código de bloque enviado por el canal
print("## COMPARACION DEL MENSAJE CODIFICADO CON EL CORREGIDO ##");
print("palabra codificada original:\t ", coded_word_original);
print("palabra erronea:\t\t ", palabra_erronea);
print("palabra corregida:\t\t ", palabra_corregida);
print("sindrome:\t\t\t ", sindrome);
# Decodificación.
palabra_decodificada = Decodificar (palabra_corregida, M);
# Comparación con el mensaje del código uniforme generado por la fuente
print("## COMPARACION DEL MENSAJE DECODIFICADO CON EL ORIGINAL ##");
print("palabra original:\t ", palabra);
print("palabra decodificada:\t ", palabra_decodificada);

## COMPARACION DEL MENSAJE CODIFICADO CON EL CORREGIDO ##
palabra codificada original:	  [1 1 0 0 0 1 0 0 0 1 1 1]
palabra erronea:		  [1 1 1 0 0 1 0 0 0 1 1 1]
palabra corregida:		  [1 1 0 0 0 1 0 0 0 1 1 1]
sindrome:			  [1 0 1 0]
## COMPARACION DEL MENSAJE DECODIFICADO CON EL ORIGINAL ##
palabra original:	  [1 1 0 0 1 0 0 1]
palabra decodificada:	  [1 1 0 0 1 0 0 1]


In [75]:

# EJEMPLO 3

# Ejemplos de prueba de envío con errores en 2 bits

# Generar mensaje a enviar
palabra = GenerarPalabraAleatoria(k);
# Codificación en código de bloque
M = CalcularMatrizM(n, k);
coded_word = Codificar (palabra, M);
coded_word_original = coded_word.copy();
# Inserción de un error de 1 bit
InsertarError2bits(coded_word);
palabra_erronea = coded_word.copy();
# Cálculo del error y corrección
H = CalcularMatrizH(n, k);
sindrome = CalcularSindrome(coded_word, H);
palabra_corregida = CorregirError(coded_word, sindrome);
# Comprobación con el mensaje del código de bloque enviado por el canal
print("## COMPARACION DEL MENSAJE CODIFICADO CON EL CORREGIDO ##");
print("palabra codificada original:\t ", coded_word_original);
print("palabra erronea:\t\t ", palabra_erronea);
print("palabra corregida:\t\t ", palabra_corregida);
print("sindrome:\t\t\t ", sindrome);
# Decodificación.
palabra_decodificada = Decodificar (palabra_corregida, M);
# Comparación con el mensaje del código uniforme generado por la fuente
print("## COMPARACION DEL MENSAJE DECODIFICADO CON EL ORIGINAL ##");
print("palabra original:\t ", palabra);
print("palabra decodificada:\t ", palabra_decodificada);

## COMPARACION DEL MENSAJE CODIFICADO CON EL CORREGIDO ##
palabra codificada original:	  [1 0 0 0 1 1 0 0 0 1 0 0]
palabra erronea:		  [1 0 1 0 1 1 0 0 0 0 0 0]
palabra corregida:		  [1 0 1 1 1 1 0 0 0 0 0 0]
sindrome:			  [1 0 0 1]
## COMPARACION DEL MENSAJE DECODIFICADO CON EL ORIGINAL ##
palabra original:	  [1 0 0 0 1 0 0 1]
palabra decodificada:	  [1 0 1 1 1 0 0 0]
